# Example: Overview of Primary Query Functionality

### Imports and Token Assignment

In [ ]:
import os
from pprint import pprint

token = os.environ['EC3_KEY']

### Querying EPDs

In [ ]:
#Getting EPD records example
from ec3 import EC3epds
ec3_epds = EC3epds(bearer_token=token, ssl_verify=False)

epd_param_dict = {"product_classes":{"EC3":"Concrete >> ReadyMix"}, "concrete_compressive_strength_at_28d__target":"3000 psi", "lightweight":True, "applicable_in":["NAFTA"]}
ec3_epds.max_records = 10 #default max records is 100
ec3_epds.only_valid = False #Default is True. You can set this to False if you want to also return expired EPDs
epd_records = ec3_epds.get_epds(params=epd_param_dict)

pprint(epd_records[0])

In [ ]:
#Get an epd by xpd uuid
epd_xpd_uuid = "ec3ftbxgp9"

epd_record = ec3_epds.get_epd_by_xpduuid(epd_xpd_uuid)
pprint(epd_record[0])

### Querying Materials

In [ ]:
#Getting material records example
from ec3 import EC3Materials
ec3_materials = EC3Materials(bearer_token=token, ssl_verify=False)

mat_param_dict = {"lightweight":True, "concrete_compressive_strength_at_28d__target":"5000 psi", "jurisdiction":"US"} #Dictonary of params and values. See EC3 Api documentation for options.

ec3_materials.return_fields = ["id", "concrete_compressive_strength_28d", "gwp"] #This can be used to limit the fields beings returned
ec3_materials.sort_by = "concrete_compressive_strength_28d" #This will sort the responses based on the field assiged to the 'sort_by' property

mat_records = ec3_materials.get_materials(return_all=True, params=mat_param_dict) #Setting return_all to True will ignore the max_records number and attempt to return all matches

pprint(mat_records[0])

In [ ]:
#Get an material by the xpd uuid of an epd
mat_record = ec3_materials.get_material_by_xpduuid(epd_xpd_uuid)
pprint(mat_record[0])